In [1]:
print("Hello World")

Hello World


In [10]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x7c901c0817d0>)

In [11]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [15]:
import re
pre_processed_text = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
pre_processed_text = [item.strip() for item in pre_processed_text if item.strip()]
print(len(pre_processed_text))

4690


In [16]:
all_words = sorted(set(pre_processed_text))
vocab_size = len(all_words)
print(vocab_size)

1130
